### Indexen

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.html

We zullen beginnen met het bekijken van Pandas `Index` objecten.

Indexen op zichzelf zijn niet zo nuttig, maar ze zijn nauw geïntegreerd met twee andere fundamentele typen in Pandas: `Series` en `DataFrame`.  Hiervoor moeten we indexen begrijpen.

In [2]:
import pandas as pd
import numpy as np

Een index is eigenlijk gewoon een verzameling van waarden.  We gaan even enkele indexen maken:

In [3]:
idx = pd.Index([10, 20, 30])

In [4]:
idx

Index([10, 20, 30], dtype='int64')

Zoals je kunt zien, heeft een index een specifiek gegevenstype - net zoals een NumPy-array (Pandas gebruikt NumPy arrays!).  Hier is het een int64 omdat het datatype van 10, 20, 30 int64 is.  

Als we gegevenstypen mengen, zal Pandas een geschikt gegevenstype vinden dat 'breed' genoeg is voor onze elementen:

In [5]:
idx = pd.Index([1, 3.14])
idx

Index([1.0, 3.14], dtype='float64')

We kunnen ook strings specificeren voor indexelementen:

In [6]:
idx = pd.Index(['element 1', 'element 2'])
idx

Index(['element 1', 'element 2'], dtype='object')

Het feit dat dtype = object betekent dat dit een generiek Python object is.  Er is namelijk geen string type, dus wordt object gebruikt. Dit is ook zo bij NumPy: als je verschillende datatypes mengt in een array, wordt ook object gebruikt.   

Wat interessant is aan indices, is dat ze zich zowel kunnen gedragen als arrays als als sets.

Laten we eerst naar de array-achtige gedragingen kijken:

In [7]:
idx = pd.Index([2, 4, 6, 8, 10])
idx

Index([2, 4, 6, 8, 10], dtype='int64')

Dit gedraagt zich als een gewone array: we kunnen de index gebruiken:

In [8]:
idx[0]

2

**Slicen** werkt ook:

In [9]:
idx[1:4]

Index([4, 6, 8], dtype='int64')

Omkeren met **extended slicing**:

In [10]:
idx[::-1]

Index([10, 8, 6, 4, 2], dtype='int64')

We kunnen gebruikmaken van **fancy indexing** (aangezien Index-arrays gebaseerd zijn op NumPy-arrays):

In [11]:
idx[[1, 3, 4]]

Index([4, 8, 10], dtype='int64')

En we kunnen ook **boolean masking** gebruiken, net zoals bij normale NumPy-arrays:

In [13]:
idx = pd.Index(['London', 'Paris', 'New York', 'Tokyo'])
idx[idx != 'Tokyo']

Index(['London', 'Paris', 'New York'], dtype='object')

Merk op dat fancy indexing of slicing **een nieuw `Index` object** retourneert, niet enkel een Python-lijst of NumPy-array.

Echter, een index is **niet** veranderlijk:

In [12]:
try:
    idx[0] = 100
except TypeError as ex:
    print('TypeError:', ex)

TypeError: Index does not support mutable operations


Indexen zijn dus immutable!

Merk op dat de waarden die we steeds hebben gebruikt binnen de vierkante haken (`[]`) bij slicing, indexing enz de **positionele indices** zijn van de elementen binnen de `Index` objecten.

`Index` objecten gedragen zich ook als sets in die zin dat we de **union en intersection** ervan kunnen vinden:

In [44]:
idx_1 = pd.Index(['a', 'b', 'c'])
idx_2 = pd.Index(['c', 'd', 'e'])

In [45]:
idx_1.intersection(idx_2)

Index(['c'], dtype='object')

In [48]:
'a' in idx_1

True

Als we verschillende gegevenstypen hebben, zal Pandas het breedste gegevenstype gebruiken dat nodig is voor het resulterende `Index` object:

In [47]:
idx_1.union(idx_2)

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

In [49]:
pd.Index([1, 2, 3]).union(pd.Index([0.1, 0.2]))

Float64Index([0.1, 0.2, 1.0, 2.0, 3.0], dtype='float64')

Pandas kiest het beste datatype voor de merge!

Pandas implementeert ook veel verschillende soorten gespecialiseerde indexen, zoals- `RangeIndex`
- `Int64Index`
- `Float64Index`
- en veel andere...



We hebben al `Int64Index` en `Float64Index` gezien:

In [50]:
pd.Index([1, 2, 3])

Int64Index([1, 2, 3], dtype='int64')

In [51]:
pd.Index([0.1, 0.2])

Float64Index([0.1, 0.2], dtype='float64')

Een range index wordt eenvoudig gemaakt met behulp van deze twee benaderingen:

Via een Python `range` object:

In [52]:
pd.Index(range(2, 10, 2))

RangeIndex(start=2, stop=10, step=2)

Of direct in Pandas:

In [53]:
pd.RangeIndex(2, 10, 2)

RangeIndex(start=2, stop=10, step=2)

We kunnen nog steeds verwijzen naar elementen binnen die index zoals we eerder hebben gezien:

In [21]:
idx = pd.RangeIndex(2, 10, 2)
idx

RangeIndex(start=2, stop=10, step=2)

In [22]:
idx[0]

2

In [23]:
idx[1:4]

RangeIndex(start=4, stop=10, step=2)

In [24]:
idx[::-1]

RangeIndex(start=8, stop=0, step=-2)

Je zult merken dat Pandas slim omgaat met de resultaten voor slices - het genereert geen volledige lijst van waarden - in plaats daarvan wordt er gewoon een andere range index gemaakt. Dit betekent dat een RangeIndex veel efficiënter kan zijn qua opslag en soms berekeningssnelheid dan een reguliere index met dezelfde expliciete waarden.

We kunnen zelfs unions en intersections op dezelfde manier behandelen:

In [54]:
idx_1 = pd.RangeIndex(0, 5)
list(idx_1)

[0, 1, 2, 3, 4]

In [55]:
idx_2 = pd.RangeIndex(4, 8)
list(idx_2)

[4, 5, 6, 7]

In [61]:
idx_1.intersection(idx_2)

RangeIndex(start=4, stop=5, step=1)

In [62]:
list(idx_1.intersection(idx_2))

[4]

In [63]:
idx_1.union(idx_2)

RangeIndex(start=0, stop=8, step=1)

In [65]:
list(idx_1.union(idx_2))

[0, 1, 2, 3, 4, 5, 6, 7]

Niet alle unions en intersections van ranges kunnen worden uitgedrukt als een nieuwe range, dus soms eindigen we met een reguliere index in plaats van een range index:

In [66]:
pd.RangeIndex(1, 10, 2).union(pd.RangeIndex(1, 10, 3))

Int64Index([1, 3, 4, 5, 7, 9], dtype='int64')

We kunnen `in` gebruiken voor het testen van de aanwezigheid van indexen in het algemeen:

In [32]:
idx_1 = pd.Index(['a', 'b', 'c'])
idx_2 = pd.RangeIndex(0, 10, 2)

In [33]:
'b' in idx_1

True

In [34]:
6 in idx_2

True

In [35]:
'x' in idx_1

False

In [36]:
1 in idx_2

False

Een laatste punt om op te merken is dat indexwaarden niet uniek hoeven te zijn:

In [37]:
idx = pd.Index([1, 1, 2, 2, 3, 3])
idx

Int64Index([1, 1, 2, 2, 3, 3], dtype='int64')